## Websearch and visitwebpage agent

In [ ]:
from smolagents import LiteLLMModel, CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool

model = LiteLLMModel(
    model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)

agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    additional_authorized_imports=["requests"]
)
system_prompt = """
You are a professional web research assistant. Your primary goal is to find accurate and up-to-date information from the internet in response to user questions.
Always keep in mind the initial question.

You have access to two tools:

- web_search: Use this to search the web and retrieve a list of relevant webpages.
- visit_webpage: Use this to visit a specific webpage URL and extract the relevant content.

When the user asks a question that requires current or external information:
- Use the web_search function to search the web using a precise, relevant query. Review the search results and identify the most credible or relevant URLs.
- Use the visit_webpage function to visit one or more of those URLs to extract accurate information.

Summarize your findings clearly and concisely for the user. If applicable, include the source of your information.

Only use the tools if the answer cannot be deduced from general knowledge or your current context. If you did a web_search, do visit_webpage also. Always prioritize accuracy and clarity in your responses.
"""

question = "What is the favourite dessert of Elisabeth II of United Kingdom?"
prompt="How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."

agent.run(prompt+system_prompt)

## Selenium agent (WIP)

In [ ]:
import argparse
from io import BytesIO
from time import sleep

import helium
from dotenv import load_dotenv
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from smolagents import CodeAgent, DuckDuckGoSearchTool, tool, LiteLLMModel
from smolagents.agents import ActionStep
from smolagents.cli import load_model
import os

os.environ["OPENAI_API_KEY"] = "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"


alfred_guest_list_request = """
I am Alfred, the butler of Wayne Manor, responsible for verifying the identity of guests at party. A superhero has arrived at the entrance claiming to be Wonder Woman, but I need to confirm if she is who she says she is.
Please search for images of Wonder Woman and generate a detailed visual description based on those images. Additionally, navigate to Wikipedia to gather key details about her appearance. With this information, I can determine whether to grant her access to the event.
"""


def parse_arguments():
    parser = argparse.ArgumentParser(description="Run a web browser automation script with a specified model.")
    parser.add_argument(
        "prompt",
        type=str,
        nargs="?",  # Makes it optional
        default=alfred_guest_list_request,
        help="The prompt to run with the agent",
    )
    parser.add_argument(
        "--model-type",
        type=str,
        default="LiteLLMModel",
        help="The model type to use (e.g., OpenAIServerModel, LiteLLMModel, TransformersModel, InferenceClientModel)",
    )
    parser.add_argument(
        "--model-id",
        type=str,
        default="gpt-4o",
        help="The model ID to use for the specified model type",
    )
    return parser.parse_args()


def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Let JavaScript animations happen before taking the screenshot
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        for previous_memory_step in agent.memory.steps:  # Remove previous screenshots from logs for lean processing
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"Captured a browser screenshot: {image.size} pixels")
        memory_step.observations_images = [image.copy()]  # Create a copy to ensure it persists, important!

    # Update observations with current URL
    url_info = f"Current url: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )
    return


@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result


@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()


@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows! This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


def initialize_driver():
    """Initialize the Selenium WebDriver."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--force-device-scale-factor=1")
    chrome_options.add_argument("--window-size=1000,1350")
    chrome_options.add_argument("--disable-pdf-viewer")
    chrome_options.add_argument("--window-position=0,0")
    return helium.start_chrome(headless=False, options=chrome_options)


def initialize_agent(model):
    """Initialize the CodeAgent with the specified model."""
    return CodeAgent(
        tools=[DuckDuckGoSearchTool(), go_back, close_popups, search_item_ctrl_f],
        model=model,
        additional_authorized_imports=["helium"],
        step_callbacks=[save_screenshot],
        max_steps=20,
        verbosity_level=2,
    )


helium_instructions = """
Use your web_search tool when you want to get Google search results.
Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!
Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"
Then you can go to pages! Don't go directly to specific urls, only use links from google search.
Code:
```py
go_to('github.com/trending')
```<end_code>
You can directly click clickable elements by inputting the text that appears on them.
Code:
```py
click("Top products")
```<end_code>
If it's a link:
Code:
```py
click(Link("Top products"))
```<end_code>
If you try to interact with an element and it's not found, you'll get a LookupError.
In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.
To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>
When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>
You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>
Proceed in several steps rather than trying to solve the task in one shot.
And at the end, only when you have your answer, return your final answer.
Code:
```py
final_answer("YOUR_ANSWER_HERE")
```<end_code>
If pages seem stuck on loading, you might have to wait, for instance `import time` and run `time.sleep(5.0)`. But don't overuse this!
To list elements on page, DO NOT try code-based element searches like 'contributors = find_all(S("ol > li"))': just look at the latest screenshot you have and read it visually, or use your tool search_item_ctrl_f.
Of course, you can act on buttons like a user would do when navigating.
After each code blob you write, you will be automatically provided with an updated screenshot of the browser and the current browser url.
But beware that the screenshot will only be taken at the end of the whole action, it won't see intermediate states.
Don't kill the browser.
When you have modals or cookie banners on screen, you should get rid of them before you can click anything else.
"""


def main():
    # Load environment variables
    # load_dotenv()

    # # Parse command line arguments
    # args = parse_arguments()

    # Initialize the model based on the provided arguments
    #model = load_model(args.model_type, args.model_id)
    # model = load_model(
    #     model_type='OpenAIServerModel', 
    #     model_id='mistralai/devstral-small:free',
    #     api_key=os.environ["OPENAI_API_KEY"],
    #     api_base='https://openrouter.ai/api/v1'
    # )

    # model = load_model(
    #     model_type='LiteLLMModel', 
    #     model_id='ollama_chat/qwen2:7b',
    #     api_base='http://127.0.0.1:11434',
    # )

    model = LiteLLMModel(
    model_id='ollama_chat/qwen2:7b',
    api_base='http://127.0.0.1:11434',
    flatten_messages_as_text=False
)

    global driver
    driver = initialize_driver()
    agent = initialize_agent(model)

    # Run the agent with the provided prompt
    agent.python_executor("from helium import *")
    prompt="How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."
    agent.run(prompt + helium_instructions)

main()


## Agent search link Google Flights

In [1]:
import asyncio
from dotenv import load_dotenv
import os

from browser_use import Agent
from langchain_anthropic import ChatAnthropic

load_dotenv()

task_system="""
You are a specialized flight search agent designed to efficiently collect and organize flight booking links from travel websites. Your expertise lies in translating user travel requirements into direct, actionable flight search URLs for comprehensive analysis.

## Core Responsibilities
- Generate precise flight search URLs based on user criteria
- Ensure links capture all specified travel parameters
- Provide URLs optimized for easy booking and comparison
- Maintain consistency in link format and structure

## Input Parameters
You will process the following user-provided information:

### Required Parameters
- **Departure**: Origin airport code, city name, or location
- **Destination**: Destination airport code, city name, or location  
- **Date**: Travel date in YYYY-MM-DD format (or date range for flexible searches)
- **Passengers**: Number and type of travelers (adults, children, infants)

### Optional Parameters (when provided)
- **Return Date**: For round-trip flights
- **Class**: Economy, Premium Economy, Business, or First Class
- **Airlines**: Preferred or excluded carriers
- **Stops**: Direct flights only, 1 stop maximum, etc.
- **Time Preferences**: Departure/arrival time windows
- **Budget Range**: Price constraints

## Search Strategy
1. **Primary Platform**: Google Flights (primary source for comprehensive results)
2. **URL Construction**: Build search URLs that include all specified parameters
3. **Parameter Encoding**: Ensure proper URL encoding for special characters and spaces
4. **Validation**: Verify links are functional and contain correct search criteria

## Output Format

### Standard Response Structure
```
**Flight Search Link:**
[Generated URL]

**Search Parameters Captured:**
- Route: [Origin] → [Destination]
- Date: [Travel Date(s)]
- Passengers: [Count and type]
- Additional filters: [Any special requirements]
```

### Example Output
```
**Flight Search Link:**
Example: https://www.google.com/travel/flights/search?tfs=CBwQAhpJEgoyMDI1LTA2LTI4Ih8KA0NERxIKMjAyNS0wNi0yOBoDWVVMKgJBQzIDODc1agwIAhIIL20vMDVxdGpyDAgDEggvbS8wNTJwNxooEgoyMDI1LTA4LTMwagwIAxIIL20vMDUycDdyDAgCEggvbS8wNXF0akABSAFwAYIBCwj___________8BmAEB&tfu=CmxDalJJZVV4VU1UTlJNWHBrYkdkQlQxZGxWVkZDUnkwdExTMHRMUzB0ZDJKaVpYY3hOVUZCUVVGQlIyaFBjRTkzU0RJeVJWZEJFZ1ZCUXpnM05Sb0xDUDNLQWhBQ0dnTkZWVkk0SEhDNy9nST0SAggAIgA
"""

task="I want to travel from Paris to Montréal from June 28th to August 30th."

async def main():
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        raise ValueError("Clé API manquante : vérifie ton .env")

    agent = Agent(
        task=task_system+task,
        llm=ChatAnthropic(
            model="claude-3-7-sonnet-20250219",
            api_key=api_key,
        ),
    )
    await agent.run()

await main()


INFO     [browser_use.telemetry.service] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
INFO     [browser_use.Agent✻70a3 on ⛶70a3.88] 🧠 Starting a browser-use agent 0.2.7 with base_model=claude-3-7-sonnet-20250219 +vision +memory extraction_model=None 


/Users/sylvainchen/Documents/Telecom/hackathon-huggingface/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     [browser_use.Agent✻70a3 on ⛶70a3.88] 🚀 Starting task: 
You are a specialized flight search agent designed to efficiently collect and organize flight booking links from travel websites. Your expertise lies in translating user travel requirements into direct, actionable flight search URLs for comprehensive analysis.

## Core Responsibilities
- Generate precise flight search URLs based on user criteria
- Ensure links capture all specified travel parameters
- Provide URLs optimized for easy booking and comparison
- Maintain consistency in link format and structure

## Input Parameters
You will process the following user-provided information:

### Required Parameters
- **Departure**: Origin airport code, city name, or location
- **Destination**: Destination airport code, city name, or location  
- **Date**: Travel date in YYYY-MM-DD format (or date range for flexible searches)
- **Passengers**: Number and type of travelers (adults, children, infants)

### Optional Parameters (when 

## Agent flights info

In [ ]:
import argparse
from io import BytesIO
from time import sleep
import json
import re
from typing import List, Dict

import helium
from dotenv import load_dotenv
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from smolagents import CodeAgent, DuckDuckGoSearchTool, tool, LiteLLMModel
from smolagents.agents import ActionStep
from smolagents.cli import load_model
import os

# Configuration de l'API
os.environ["OPENAI_API_KEY"] = "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"

# Prompt principal pour l'agent de récupération de liens de vols
flight_links_request = """
I am a flight booking assistant. I need to analyze Google Flights search results and extract direct booking links for flights.

Your task is to:
1. Navigate to the provided Google Flights search URL
2. Wait for the page to load completely and show flight results
3. Find and extract the booking links for the first 5 flight options
4. For each flight, collect:
   - The airline name
   - Departure and arrival times
   - Price
   - Direct booking link (URL)
   - Flight duration
   - Number of stops (if any)

Please be methodical and wait for pages to load properly. Focus on finding clickable booking buttons or links that lead to airline booking pages.
"""

def parse_arguments():
    parser = argparse.ArgumentParser(description="Extract flight booking links from Google Flights search results.")
    parser.add_argument(
        "flights_url",
        type=str,
        help="The Google Flights search URL to analyze"
    )
    parser.add_argument(
        "--model-type",
        type=str,
        default="LiteLLMModel",
        help="The model type to use"
    )
    parser.add_argument(
        "--model-id",
        type=str,
        default="gpt-4o",
        help="The model ID to use"
    )
    parser.add_argument(
        "--max-flights",
        type=int,
        default=5,
        help="Maximum number of flights to extract (default: 5)"
    )
    return parser.parse_args()

def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Laisser le temps aux animations JavaScript
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        # Nettoyer les anciens screenshots pour optimiser la mémoire
        for previous_memory_step in agent.memory.steps:
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"📸 Screenshot capturé: {image.size} pixels")
        memory_step.observations_images = [image.copy()]

    # Mettre à jour les observations avec l'URL actuelle
    url_info = f"Current URL: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )
    return

@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Recherche du texte sur la page actuelle via Ctrl + F et va au nième résultat.
    Args:
        text: Le texte à rechercher
        nth_result: Quelle occurrence choisir (défaut: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Résultat n°{nth_result} non trouvé (seulement {len(elements)} résultats)")
    result = f"Trouvé {len(elements)} résultats pour '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focus sur l'élément {nth_result} de {len(elements)}"
    return result

@tool
def go_back() -> None:
    """Retourne à la page précédente."""
    driver.back()

@tool
def close_popups() -> str:
    """
    Ferme les modales ou pop-ups visibles sur la page.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    return "Pop-ups fermés"

@tool
def wait_for_page_load(seconds: int = 3) -> str:
    """
    Attend que la page se charge complètement.
    Args:
        seconds: Nombre de secondes à attendre
    """
    sleep(seconds)
    return f"Attente de {seconds} secondes terminée"

@tool
def extract_flight_info() -> str:
    """
    Extrait les informations des vols visibles sur la page Google Flights.
    Retourne un JSON avec les détails des vols.
    """
    try:
        # Rechercher les éléments de vol sur la page
        flight_elements = driver.find_elements(By.CSS_SELECTOR, '[data-testid="flight-card"], .pIav2d, .yR1fYc')
        
        if not flight_elements:
            # Essayer d'autres sélecteurs possibles
            flight_elements = driver.find_elements(By.CSS_SELECTOR, '[role="listitem"], .JMc5Xc')
        
        flights_info = []
        
        for i, element in enumerate(flight_elements[:5]):  # Limiter aux 5 premiers
            try:
                # Extraire les informations de chaque vol
                flight_data = {
                    "flight_number": i + 1,
                    "element_text": element.text[:200] if element.text else "No text",
                    "clickable": element.is_enabled() and element.is_displayed()
                }
                flights_info.append(flight_data)
            except Exception as e:
                print(f"Erreur lors de l'extraction du vol {i+1}: {e}")
                continue
        
        return json.dumps(flights_info, indent=2)
    
    except Exception as e:
        return f"Erreur lors de l'extraction: {str(e)}"

@tool
def get_current_url() -> str:
    """Retourne l'URL actuelle du navigateur."""
    return driver.current_url

@tool
def click_flight_option(flight_number: int) -> str:
    """
    Clique sur une option de vol spécifique.
    Args:
        flight_number: Numéro du vol à cliquer (1-5)
    """
    try:
        flight_elements = driver.find_elements(By.CSS_SELECTOR, '[data-testid="flight-card"], .pIav2d, .yR1fYc')
        
        if not flight_elements:
            flight_elements = driver.find_elements(By.CSS_SELECTOR, '[role="listitem"], .JMc5Xc')
        
        if flight_number <= len(flight_elements):
            element = flight_elements[flight_number - 1]
            driver.execute_script("arguments[0].click();", element)
            sleep(2)  # Attendre le chargement
            return f"Cliqué sur le vol {flight_number}"
        else:
            return f"Vol {flight_number} non trouvé (seulement {len(flight_elements)} vols disponibles)"
    
    except Exception as e:
        return f"Erreur lors du clic sur le vol {flight_number}: {str(e)}"

def initialize_driver():
    """Initialise le WebDriver Selenium."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--force-device-scale-factor=1")
    chrome_options.add_argument("--window-size=1400,1000")
    chrome_options.add_argument("--disable-pdf-viewer")
    chrome_options.add_argument("--window-position=0,0")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    return helium.start_chrome(headless=False, options=chrome_options)

def initialize_agent(model):
    """Initialise le CodeAgent avec le modèle spécifié."""
    return CodeAgent(
        tools=[
            DuckDuckGoSearchTool(), 
            go_back, 
            close_popups, 
            search_item_ctrl_f,
            wait_for_page_load,
            extract_flight_info,
            get_current_url,
            click_flight_option
        ],
        model=model,
        additional_authorized_imports=["helium", "time", "json", "re"],
        step_callbacks=[save_screenshot],
        max_steps=25,
        verbosity_level=2,
    )

# Instructions spécifiques pour helium et Google Flights
helium_instructions = """
Use helium to navigate websites and interact with Google Flights.
We've already imported helium: "from helium import *"

Key helium commands:
- go_to('url'): Navigate to a URL
- click("text"): Click on text or button
- click(Link("text")): Click on a link
- scroll_down(num_pixels=800): Scroll down
- scroll_up(num_pixels=800): Scroll up
- wait_until(lambda: Text('text').exists()): Wait for element

For Google Flights specifically:
1. First go to the provided Google Flights URL
2. Wait for the page to load completely (use wait_for_page_load(5))
3. Close any popups or cookie banners with close_popups()
4. Look for flight results and extract information
5. Try to click on flight options to get booking links
6. Be patient - Google Flights loads dynamically

Always take screenshots after each action to see the current state.
When you find booking buttons or "Select" buttons, click them to get airline booking links.

Important: Google Flights often has multiple steps:
1. Search results page
2. Flight details page after clicking a flight
3. Booking page or redirect to airline website

Your goal is to collect 5 flight booking links with their details.

Use your tools:
- extract_flight_info(): To get flight information from the current page
- click_flight_option(number): To click on a specific flight
- wait_for_page_load(seconds): To wait for page loading

Code example:
```py
go_to('https://www.google.com/travel/flights/...')
wait_for_page_load(5)
close_popups()
extract_flight_info()
```

At the end, provide a final answer with the collected flight information and booking links.
"""

def main():
    # Charger les variables d'environnement
    load_dotenv()

    # Parser les arguments de ligne de commande
    args = parse_arguments()

    # Initialiser le modèle
    model = LiteLLMModel(
        model_id='ollama_chat/qwen2:7b',
        api_base='http://127.0.0.1:11434',
        flatten_messages_as_text=False
    )

    global driver
    driver = initialize_driver()
    agent = initialize_agent(model)

    # Exécuter l'agent avec le prompt
    agent.python_executor("from helium import *")
    
    # Construire le prompt complet
    full_prompt = f"""
    {flight_links_request}
    
    Google Flights URL to analyze: {args.flights_url}
    
    Extract information and booking links for up to {args.max_flights} flights.
    
    {helium_instructions}
    """
    
    print(f"🛫 Démarrage de l'analyse des vols...")
    print(f"🔗 URL: {args.flights_url}")
    print(f"📊 Nombre de vols à extraire: {args.max_flights}")
    print("=" * 80)
    
    try:
        result = agent.run(full_prompt)
        print("=" * 80)
        print("✅ Analyse terminée!")
        print(f"📄 Résultat: {result}")
    except Exception as e:
        print(f"❌ Erreur lors de l'exécution: {str(e)}")
    finally:
        # Fermer le navigateur
        try:
            driver.quit()
        except:
            pass

if __name__ == "__main__":
    main()

In [ ]:
import argparse
from io import BytesIO
from time import sleep

import helium
from dotenv import load_dotenv
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from smolagents import CodeAgent, DuckDuckGoSearchTool, tool, LiteLLMModel
from smolagents.agents import ActionStep
from smolagents.cli import load_model
import os

os.environ["OPENAI_API_KEY"] = "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"


alfred_guest_list_request = """
I am Alfred, the butler of Wayne Manor, responsible for verifying the identity of guests at party. A superhero has arrived at the entrance claiming to be Wonder Woman, but I need to confirm if she is who she says she is.
Please search for images of Wonder Woman and generate a detailed visual description based on those images. Additionally, navigate to Wikipedia to gather key details about her appearance. With this information, I can determine whether to grant her access to the event.
"""


def parse_arguments():
    parser = argparse.ArgumentParser(description="Run a web browser automation script with a specified model.")
    parser.add_argument(
        "prompt",
        type=str,
        nargs="?",  # Makes it optional
        default=alfred_guest_list_request,
        help="The prompt to run with the agent",
    )
    parser.add_argument(
        "--model-type",
        type=str,
        default="LiteLLMModel",
        help="The model type to use (e.g., OpenAIServerModel, LiteLLMModel, TransformersModel, InferenceClientModel)",
    )
    parser.add_argument(
        "--model-id",
        type=str,
        default="gpt-4o",
        help="The model ID to use for the specified model type",
    )
    return parser.parse_args()


def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Let JavaScript animations happen before taking the screenshot
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        for previous_memory_step in agent.memory.steps:  # Remove previous screenshots from logs for lean processing
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"Captured a browser screenshot: {image.size} pixels")
        memory_step.observations_images = [image.copy()]  # Create a copy to ensure it persists, important!

    # Update observations with current URL
    url_info = f"Current url: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )
    return


@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result


@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()


@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows! This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


def initialize_driver():
    """Initialize the Selenium WebDriver."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--force-device-scale-factor=1")
    chrome_options.add_argument("--window-size=1000,1350")
    chrome_options.add_argument("--disable-pdf-viewer")
    chrome_options.add_argument("--window-position=0,0")
    return helium.start_chrome(headless=False, options=chrome_options)


def initialize_agent(model):
    """Initialize the CodeAgent with the specified model."""
    return CodeAgent(
        tools=[DuckDuckGoSearchTool(), go_back, close_popups, search_item_ctrl_f],
        model=model,
        additional_authorized_imports=["helium"],
        step_callbacks=[save_screenshot],
        max_steps=20,
        verbosity_level=2,
    )


helium_instructions = """
Use your web_search tool when you want to get Google search results.
Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!
Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"
Then you can go to pages! Don't go directly to specific urls, only use links from google search.
Code:
```py
go_to('github.com/trending')
```<end_code>
You can directly click clickable elements by inputting the text that appears on them.
Code:
```py
click("Top products")
```<end_code>
If it's a link:
Code:
```py
click(Link("Top products"))
```<end_code>
If you try to interact with an element and it's not found, you'll get a LookupError.
In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.
To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>
When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>
You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>
Proceed in several steps rather than trying to solve the task in one shot.
And at the end, only when you have your answer, return your final answer.
Code:
```py
final_answer("YOUR_ANSWER_HERE")
```<end_code>
If pages seem stuck on loading, you might have to wait, for instance `import time` and run `time.sleep(5.0)`. But don't overuse this!
To list elements on page, DO NOT try code-based element searches like 'contributors = find_all(S("ol > li"))': just look at the latest screenshot you have and read it visually, or use your tool search_item_ctrl_f.
Of course, you can act on buttons like a user would do when navigating.
After each code blob you write, you will be automatically provided with an updated screenshot of the browser and the current browser url.
But beware that the screenshot will only be taken at the end of the whole action, it won't see intermediate states.
Don't kill the browser.
When you have modals or cookie banners on screen, you should get rid of them before you can click anything else.
"""


def main():
    # Load environment variables
    # load_dotenv()

    # # Parse command line arguments
    # args = parse_arguments()

    # Initialize the model based on the provided arguments
    #model = load_model(args.model_type, args.model_id)
    # model = load_model(
    #     model_type='OpenAIServerModel', 
    #     model_id='mistralai/devstral-small:free',
    #     api_key=os.environ["OPENAI_API_KEY"],
    #     api_base='https://openrouter.ai/api/v1'
    # )

    # model = load_model(
    #     model_type='LiteLLMModel', 
    #     model_id='ollama_chat/qwen2:7b',
    #     api_base='http://127.0.0.1:11434',
    # )

    model = LiteLLMModel(
    model_id='ollama_chat/qwen2:7b',
    api_base='http://127.0.0.1:11434',
    flatten_messages_as_text=False
)

    global driver
    driver = initialize_driver()
    agent = initialize_agent(model)

    # Run the agent with the provided prompt
    agent.python_executor("from helium import *")
    prompt="How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."
    agent.run(prompt + helium_instructions)

main()
